# Step 1. 데이터 다운로드

data 폴더에 aiffel에서 받은 데이터 파일을 저장해둠. 

# Step 2. 데이터 읽어오기


In [2]:
import glob
import os

txt_file_path = './data/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 4: illegal multibyte sequence

"UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 4: illegal multibyte sequence"

코덱 읽기 오류가 발생했는데 이는 아래와 같이 코드를 수정하면 해결된다. 

In [15]:
import glob
import os

txt_file_path = './data/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, 'rt', encoding='UTF8') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:30])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough", 'Had some bad love', "So fast love is all that I've got on my mind Ooh, ooh", 'Ooh, ooh Looking for some affirmation', 'Made my way into the sun', 'My friends got their ladies', "And they're all having babies", "I just wanna have some fun I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough", "I've had some bad love", "So fast love is all that I've got on my mind Ooh, ooh", 'Baby, baby'

정상적으로 데이터를 읽어온 것을 확인할 수 있다. 

# Step 3. 데이터 정제


In [4]:
# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes"]


여기서 볼 수 있는 건 축약어로 don't라던가 Let's 등의 표현이 자주 보인다는 점이다. 따라서 기존 코드에 ' <- 이 작은따옴표는 정제할 때 추가로 제외 시켜야한다.  

In [7]:
!pip install tensorflow

  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.1.1-py2.py3-none-any.whl (146 kB)
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=04fe77b3551eefc87138d278ad2c7b3cb50333eee7bd9ea03d7d657f724cbd11
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor


In [37]:
import os, re 
import numpy as np
import tensorflow as tf

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿,'가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있다.
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?'.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    
    # bruce-springsteen.txt에서 [Verse] 와 [Chorus]라는 구분자가 보임. 이는 문장이 아니므로 삭제함.
    if "verse" in sentence:
        sentence = sentence.replace("verse", "")
    if "chorus" in sentence:
        sentence = sentence.replace("chorus", "")
        
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample's        sentence."))

<start> this is sample's sentence . <end>


작은따옴표도 제외하고 잘 필터링 되는 것을 확인할 수 있다.

In [38]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 30개만 확인해보죠
corpus[:30]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 "<start> baby , can't you read the signs ? i won't bore you with the details , baby <end>",
 "<start> i don't even wanna waste your time <end>",
 "<start> let's just say that maybe <end>",
 '<start> you could help me ease my mind <end>',
 "<start> i ain't mr . right but if you're looking for fast love <end>",
 "<start> if that's love in your eyes <end>",
 "<start> it's more than enough <end>",
 '<start> had some bad love <end>',
 "<start> so fast love is all that i've got on my mind ooh , ooh <end>",
 '<start> ooh , ooh looking for some affirmation <end>',
 '<start> made my way into the sun <end>',
 '<start> my friends got their ladies <end>',
 "<start> and they're all having babies <end>",
 "<start> i just wanna have some fun i won't bore you with the details , baby <end>",
 "<start> i don't even wanna waste your time <end>",
 "<start> let's just 

In [40]:
len(corpus)

175749

175749 개의 정제된 문장이 준비되었다. 

우리는 여기에 너무 긴 문장은 제외할 것이다. 
- 여기서 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외할 것이다.
- 단어장 크기는 12000단어이다.

In [41]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    
    for num in tensor:
        if len(num) >= 17: # <start>, <end> 포함하여 17단어 이상(빼면 15단어 이상) 
            tensor = np.delete(tensor, num) # 너무 긴 문장은 삭제
            break;
           
    
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

C:\Users\USER\.conda\envs\python39\lib\site-packages\numpy\lib\function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


[[  2 310  26 ...   0   0   0]
 [  2 226  12 ...   0   0   0]
 [  2  48 121 ...   0   0   0]
 ...
 [  2  25  71 ...   0   0   0]
 [  2  38  23 ...   0   0   0]
 [  2  25  71 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x0000017794807340>


In [42]:
print(len(tensor))

175734


길이가 15단어 이상인 문장은 삭제되어 175749개 -> 175734개의 문장으로 정제 되었다.

In [43]:
# 어떻게 토큰이 구성되었는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 35: break # 35번까지만 확인

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : the
6 : i
7 : you
8 : and
9 : to
10 : a
11 : me
12 : my
13 : it
14 : in
15 : that
16 : on
17 : of
18 : .
19 : your
20 : i'm
21 : like
22 : all
23 : is
24 : be
25 : we
26 : for
27 : up
28 : so
29 : with
30 : know
31 : love
32 : just
33 : but
34 : don't
35 : no


In [31]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  311   26   94 4923    3    0    0    0    0    0    0    0    0]
[ 311   26   94 4923    3    0    0    0    0    0    0    0    0    0]


토큰 하나를 잘라내서 길이가 14이다.

In [32]:
#데이터 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# Step 4. 평가 데이터셋 분리

 단어장의 크기는 12,000 이상 으로 설정! 총 데이터의 20% 를 평가 데이터셋으로 사용

In [34]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=1b01e1927bf743b3aa210436e9817d9785685ebec55ec879b8bea087037ea19a
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [49]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          train_size = 0.7) # 30%를 평가 데이터셋으로 사용하므로 0.7

In [50]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (123013, 14)
Target Train: (123013, 14)


12만여개의 데이터를 기준으로 학습을 진행한다. enc_train과 dec_train이 x_train, y_train에 대응한다.

# Step 5. 인공지능 만들기

In [44]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [45]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-5.14143358e-05, -2.59825552e-04,  2.56037110e-05, ...,
          3.20734223e-04, -4.02191683e-04, -1.18090131e-04],
        [ 2.31851736e-05, -5.92685537e-04, -1.21616715e-04, ...,
          5.20093192e-04, -5.27742552e-04, -4.30101005e-04],
        [ 1.17455857e-04, -6.96399598e-04, -2.09752790e-04, ...,
          6.61058351e-04, -6.51894952e-04, -7.51067069e-04],
        ...,
        [-2.64651142e-04, -1.26127643e-03,  1.23470696e-03, ...,
          7.84380420e-04, -3.94543691e-04,  6.78902376e-04],
        [-3.82092316e-04, -1.45593414e-03,  1.46202208e-03, ...,
          8.07594857e-04,  1.07009357e-04,  1.15199992e-03],
        [-4.67140751e-04, -1.63161859e-03,  1.64141133e-03, ...,
          8.76647537e-04,  5.64257498e-04,  1.58145372e-03]],

       [[-5.14143358e-05, -2.59825552e-04,  2.56037110e-05, ...,
          3.20734223e-04, -4.02191683e-04, -1.18090131e-04],
        [-2.02648516e-04, -3.71801871e-04,  2

In [46]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [47]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10) # 노트북으로 짆행시 시간이 오래 결려 PC 환경에서 진행하도록함. 

Epoch 1/30
 12/686 [..............................] - ETA: 1:32:06 - loss: 6.6346

KeyboardInterrupt: 

In [48]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he")

# 마무리

